In [ ]:
from diagrams.aws.integration import Eventbridge
from diagrams.aws.compute import LambdaFunction
from diagrams.aws.storage import SimpleStorageServiceS3BucketWithObjects
from diagrams.aws.database import RDSInstance
from diagrams.aws.compute import Fargate


with Diagram("Event FeedBack Handler", show=True):
    event_inputs =  EC2("Backend Service")
    with Cluster("Cleaning"):
      lambda_data_extract = Fargate("Data extractor")

      lambda_image_processer = Fargate("Image transform")
      lambda_data_cleaning = Fargate("Comment transform")

    s3_object_images = SimpleStorageServiceS3BucketWithObjects("GIF comments")
    rds_comments = RDSInstance("comments db")

    event_inputs >> lambda_data_extract
    lambda_data_extract >> lambda_data_cleaning
    lambda_data_extract >> lambda_image_processer
    lambda_image_processer >> s3_object_images
    lambda_data_cleaning >> rds_comments




In [ ]:
from diagrams.aws.management import CloudwatchEventTimeBased
from diagrams.aws.compute import LambdaFunction
from diagrams.aws.storage import SimpleStorageServiceS3BucketWithObjects
from diagrams.aws.database import RDSInstance
from diagrams.aws.analytics import GlueCrawlers
from diagrams.aws.analytics import Glue



with Diagram("Batch FeedBack Handler", show=True):
    event_alarm =  CloudwatchEventTimeBased("Schecduled Trigger")

    unprocessed_s3_object_images = SimpleStorageServiceS3BucketWithObjects("Unprocessed GIF comments")
    processed_s3_object_images = SimpleStorageServiceS3BucketWithObjects("processed GIF")

    unprocessed_comments = RDSInstance("Unprocessed comments")
    processed_comments = RDSInstance("Processed comments")

    lambda_data_starter = LambdaFunction("Glue trigger")

    s3crawler = GlueCrawlers("s3 crawler")
    rdscrawler = GlueCrawlers("rds crawler")

    glue_processing_image_job = Glue("image processing job")
    glue_processing_text_job = Glue("text processing job")

    event_alarm >> lambda_data_starter
    lambda_data_starter >> s3crawler
    lambda_data_starter >> rdscrawler
    unprocessed_s3_object_images >> s3crawler
    unprocessed_comments >> rdscrawler
    s3crawler >> glue_processing_image_job
    rdscrawler >> glue_processing_text_job
    glue_processing_image_job >> processed_s3_object_images
    glue_processing_text_job >> processed_comments




In [ ]:
from diagrams.aws.management import CloudwatchEventTimeBased
from diagrams.aws.compute import LambdaFunction
from diagrams.aws.storage import SimpleStorageServiceS3BucketWithObjects
from diagrams.aws.database import RDSInstance
from diagrams.aws.analytics import GlueCrawlers
from diagrams.aws.analytics import Glue
from diagrams.aws.database import DynamodbItems
from diagrams.aws.ml import SagemakerModel
from diagrams import Cluster, Diagram
# from diagrams.aws.analytics import Redshift
from diagrams.aws.ml import SagemakerNotebook

with Diagram("Batch Inference Sentiment", show=True):

    event_alarm =  CloudwatchEventTimeBased("Schecduled Trigger")


    lambda_data_starter = LambdaFunction("Glue trigger")
    processed_comments = RDSInstance("Processed comments")
    processed_s3_object_images = SimpleStorageServiceS3BucketWithObjects("processed GIF")

    with Cluster("Aws Step Functions"):


      productcrawler = GlueCrawlers("Product crawler")

      gifcrawler = GlueCrawlers("gif crawler")
      rdscrawler = GlueCrawlers("rds crawler")

      # taining_s3_object_images = SimpleStorageServiceS3BucketWithObjects("Inference GIF")

      training_comments = DynamodbItems("comments features store")

      sagemaker_aggregated_job = SagemakerModel("Topic Score generator")

      # lambda_gif_processer = LambdaFunction("Process Gif features")
      glue_processing_image_job = Glue("gif transformer")
      glue_processing_text_job = Glue("text features generator")

      # lambda_text_preparer = LambdaFunction("Text Pre-process")

      lambda_sentiment_preparer = Glue("Multi modal job")


      sagemaker_sentiment_job_gif = SagemakerNotebook("GIF Model")

      glue_sentiment_post_preparer = Glue("Post processer")




      feature_store_gifs = SimpleStorageServiceS3BucketWithObjects("GIF feature store")

      productcrawler >> gifcrawler
      productcrawler >> rdscrawler

      gifcrawler >> glue_processing_image_job
      rdscrawler >> glue_processing_text_job

      glue_processing_image_job >> sagemaker_sentiment_job_gif
      glue_processing_text_job >> training_comments

      # taining_s3_object_images >> sagemaker_sentiment_job_gif

      sagemaker_sentiment_job_gif >> feature_store_gifs

      feature_store_gifs >> lambda_sentiment_preparer

      training_comments >> lambda_sentiment_preparer

      lambda_sentiment_preparer >> sagemaker_aggregated_job

      sagemaker_aggregated_job >> glue_sentiment_post_preparer


    aggregated_store = SimpleStorageServiceS3BucketWithObjects("Aggregated feature store")
    rdscrawler << processed_comments
    processed_s3_object_images >> gifcrawler
    processed_sentiment_db = RDSInstance("Sentiment DB")
    event_alarm >> lambda_data_starter
    lambda_data_starter >> productcrawler

    glue_sentiment_post_preparer >> processed_sentiment_db
    sagemaker_aggregated_job >> aggregated_store



In [ ]:
from diagrams.aws.management import CloudwatchEventTimeBased
from diagrams.aws.compute import LambdaFunction
from diagrams.aws.storage import SimpleStorageServiceS3BucketWithObjects
from diagrams.aws.database import RDSInstance
from diagrams.aws.analytics import GlueCrawlers
from diagrams.aws.analytics import Glue
from diagrams.aws.database import DynamodbItems
from diagrams.aws.ml import SagemakerModel
from diagrams import Cluster, Diagram
# from diagrams.aws.analytics import Redshift
from diagrams.aws.ml import SagemakerNotebook
from diagrams.aws.compute import Fargate

with Diagram("Architecture Diagram", show=True):
    event_alarm1 =  CloudwatchEventTimeBased("Schecduled Trigger")

    unprocessed_s3_object_images = SimpleStorageServiceS3BucketWithObjects("Unprocessed GIF comments")
    processed_s3_object_images = SimpleStorageServiceS3BucketWithObjects("processed GIF")

    unprocessed_comments = RDSInstance("Unprocessed comments")
    processed_comments = RDSInstance("Processed comments")

    lambda_data_starter1 = LambdaFunction("Glue trigger")
    with Cluster("batch clean"):

      s3crawler = GlueCrawlers("s3 crawler")
      rdscrawler = GlueCrawlers("rds crawler")

      glue_processing_image_job = Glue("image processing job")
      glue_processing_text_job = Glue("text processing job")

    event_alarm1 >> lambda_data_starter1
    lambda_data_starter1 >> s3crawler
    lambda_data_starter1 >> rdscrawler
    unprocessed_s3_object_images >> s3crawler
    unprocessed_comments >> rdscrawler
    s3crawler >> glue_processing_image_job
    rdscrawler >> glue_processing_text_job
    glue_processing_image_job >> processed_s3_object_images
    glue_processing_text_job >> processed_comments


    event_alarm =  CloudwatchEventTimeBased("Schecduled Trigger")

    event_inputs =  EC2("Backend Service")
    with Cluster("Cleaning"):
      lambda_data_extract = Fargate("Data extractor")

      lambda_image_processer = Fargate("Image transform")
      lambda_data_cleaning = Fargate("Comment transform")

    # s3_object_images = SimpleStorageServiceS3BucketWithObjects("GIF comments")
    # rds_comments = RDSInstance("comments db")

    event_inputs >> lambda_data_extract
    lambda_data_extract >> lambda_data_cleaning
    lambda_data_extract >> lambda_image_processer
    lambda_image_processer >> processed_s3_object_images
    lambda_data_cleaning >> processed_comments

    lambda_data_starter = LambdaFunction("Glue trigger")
    # processed_comments = RDSInstance("Processed comments")
    # processed_s3_object_images = SimpleStorageServiceS3BucketWithObjects("processed GIF")

    with Cluster("Aws Step Functions"):


      productcrawler = GlueCrawlers("Product crawler")

      gifcrawler = GlueCrawlers("gif crawler")
      rdscrawler1 = GlueCrawlers("rds crawler")

      # taining_s3_object_images = SimpleStorageServiceS3BucketWithObjects("Inference GIF")

      training_comments = DynamodbItems("comments features store")

      sagemaker_aggregated_job = SagemakerModel("Topic Score generator")

      # lambda_gif_processer = LambdaFunction("Process Gif features")
      glue_processing_image_job = Glue("gif transformer")
      glue_processing_text_job = Glue("text features generator")

      # lambda_text_preparer = LambdaFunction("Text Pre-process")

      lambda_sentiment_preparer = Glue("Multi modal job")


      sagemaker_sentiment_job_gif = SagemakerNotebook("GIF Model")

      glue_sentiment_post_preparer = Glue("Post processer")




      feature_store_gifs = SimpleStorageServiceS3BucketWithObjects("GIF feature store")

      productcrawler >> gifcrawler
      productcrawler >> rdscrawler1

      gifcrawler >> glue_processing_image_job
      rdscrawler1 >> glue_processing_text_job

      glue_processing_image_job >> sagemaker_sentiment_job_gif
      glue_processing_text_job >> training_comments

      # taining_s3_object_images >> sagemaker_sentiment_job_gif

      sagemaker_sentiment_job_gif >> feature_store_gifs

      feature_store_gifs >> lambda_sentiment_preparer

      training_comments >> lambda_sentiment_preparer

      lambda_sentiment_preparer >> sagemaker_aggregated_job

      sagemaker_aggregated_job >> glue_sentiment_post_preparer


    aggregated_store = SimpleStorageServiceS3BucketWithObjects("Aggregated feature store")
    rdscrawler1 << processed_comments
    processed_s3_object_images >> gifcrawler
    processed_sentiment_db = RDSInstance("Sentiment DB")
    event_alarm >> lambda_data_starter
    lambda_data_starter >> productcrawler

    glue_sentiment_post_preparer >> processed_sentiment_db
    sagemaker_aggregated_job >> aggregated_store

